In [1]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [2]:
import json
import multiprocessing
import random
import warnings
from collections import Counter
from copy import deepcopy
from typing import List, Set, Tuple

import matplotlib.pyplot as plt
import numpy as np
import plyvel
import polars as pl
import gc
import bz2
from collections import Counter
from glob import glob
from multiprocessing import Pool
import yaml
from tqdm import tqdm

import whoosh_utils
from const import INF, KEY2QUERY, NUM_CPU, ALL_KEYS
from db import CpcToken2RangeDB, SingleTokenDB, TokinezedDB
from solver import HitBlock, SimulatedAnnealing, State
from utils import compute_ap, evaluate, save_list_bz2, load_list_bz2

from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base
from penguinml.utils.timer import Timer


warnings.filterwarnings("ignore")
MODEL_NAME = "baseline"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))

init_logger("log.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


2024-06-22 09:09:43.673889: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 09:09:43.762708: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 09:09:44.382011: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

In [3]:
!rm -r patent-data
!mkdir patent-data

In [4]:
files = glob("data/*.bz2")
len(files)

251920

In [6]:
import gzip


bs = NUM_CPU * 10


def load(file):
    try:
        data = load_list_bz2(file)
    except:
        return []
    cpc = file.split("/")[-1].split(".")[0].replace("-", "/")

    datas = []
    for d in data:
        center, token, inner, outer = d
        datas.append((center, json.dumps([cpc, token, inner, outer]) + "\n"))
    return datas


def save(args):
    center, text = args
    compressed_data = bz2.compress(text.encode("utf-8"))
    with open(f"patent-data/{center}.bz2", "ab") as f:
        f.write(compressed_data)


for start in tqdm(range(0, len(files), bs)):
    end = start + bs
    batch_files = files[start:end]

    with Pool(NUM_CPU) as p:
        ret = p.map(load, batch_files)

    buffer = {}
    for data in ret:
        for center, txt in data:
            if center not in buffer:
                buffer[center] = ""
            buffer[center] += txt

    with Pool(NUM_CPU) as p:
        p.map(save, buffer.items())

100%|██████████| 840/840 [16:55:01<00:00, 72.50s/it]   
